## Load modules

In [307]:
import os, requests, json, ftplib
import pandas as pd

## Define working directory

In [308]:
PATH = ""
os.chdir(PATH)

## Define constants

In [333]:
API_TOKEN = ""
NO_TRANSACTIONS = 10000

# Request items

In [310]:
url = "http://fenixweb.net:3300/api/v2/"+API_TOKEN+"/items"
response = json.loads(requests.get(url).text)

assert response["code"] == 200, "Connection error."

# Load json into dataframe
items = pd.DataFrame(response["res"])

# Set id as index
items.set_index("id", inplace=True)

items["datetime"] = pd.to_datetime("now")
items["interval"] = 2

items.head()

,allow_sell,category,cons,craft_pnt,craftable,critical,description,dragon_power,estimate,name,power,power_armor,power_shield,rarity,rarity_name,reborn,value,datetime,interval
id,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,Un foglio di carta senza alcun significato,0,480,Carta,0,0,0,C,Comuni,1,410,2018-02-03 17:17:43,2
2,1,0,0,0,0,0,Un bastoncino di legno,0,439,Bastoncino,0,0,0,C,Comuni,1,407,2018-02-03 17:17:43,2
3,1,0,0,0,0,0,Utile per collegare due oggetti,0,423,Colla,0,0,0,C,Comuni,1,412,2018-02-03 17:17:43,2
5,1,0,0,0,0,0,Cordino poco resistente per collegare due oggetti,0,455,Cordino Fragile,0,0,0,C,Comuni,1,412,2018-02-03 17:17:43,2
7,1,4,1,0,0,0,Una piccola pietra molto leggera,0,404,Pietra Piccola,0,0,0,C,Comuni,1,404,2018-02-03 17:17:43,2


## Save items to file

In [311]:
items.to_csv("data/items.csv")

# Request market transactions

In [334]:
url = "http://fenixweb.net:3300/api/v2/"+API_TOKEN+"/history/market_direct?limit="+str(NO_TRANSACTIONS)
response = json.loads(requests.get(url).text)

assert response["code"] == 200, "Connection error."

transactions = pd.DataFrame(response["res"])
transactions.drop(columns=["buyer", "from_nick", "id", "name", "to_nick", "type"], inplace=True)

transactions.head()

,item_id,price,time
0,654,280000,2018-02-03T18:08:28.000Z
1,143,2840,2018-02-03T18:07:46.000Z
2,407,9145,2018-02-03T18:06:00.000Z
3,323,10889,2018-02-03T18:05:57.000Z
4,55,2900,2018-02-03T18:05:55.000Z


# Cleaning transactions

In [335]:
#TODO: Develop cleaning process
transactions = transactions.merge(items[["estimate", "rarity", "value"]], left_on="item_id", right_index=True, how="left")
# Remove items sold at the base price except for C type items
transactions = transactions.query("rarity != 'C' | price != value")
# Remove items with price over 5 times the estimate except for U type
transactions = transactions.query("rarity != 'U' | price < estimate*5")

# Drop estimate and values columns
transactions.drop(columns=["estimate", "rarity", "value"], inplace=True)

# Set datetime as index
transactions.time = pd.to_datetime(transactions.time, format="%Y-%m-%dT%H:%M:%S.%fZ")
transactions.set_index("time", inplace=True)

transactions.head()

(9614, 6)


,item_id,price
time,,
2018-02-03 18:08:28,654,280000
2018-02-03 18:07:46,143,2840
2018-02-03 18:06:00,407,9145
2018-02-03 18:05:57,323,10889
2018-02-03 18:05:55,55,2900


In [330]:
# Resample transactions
resampled_transactions = transactions.groupby(by=["item_id"]).resample("2D")
#resampled_transactions.drop(columns="item_id", inplace=True)

In [315]:
# TODO: optimize
numerosity = resampled_transactions.count()
numerosity.rename(columns={"price":"numerosity"}, inplace=True)
numerosity.drop(columns="item_id", inplace=True)
numerosity = numerosity.groupby(level=0).tail(1)
numerosity.index = numerosity.index.droplevel("time")

numerosity.head()

,numerosity
item_id,
1,3
2,1
3,2
4,76
5,88


In [316]:
mean = resampled_transactions.mean()
mean.rename(columns={"price":"mean"}, inplace=True)
mean.drop(columns="item_id", inplace=True)
mean = mean.groupby(level=0).tail(1)
mean.index = mean.index.droplevel("time")
mean = mean.groupby(level=0).tail(1)

mean.head()

,mean
item_id,
1,3.345733e+07
2,4.390000e+02
3,1.225000e+03
4,1.690434e+03
5,1.581818e+03


In [317]:
std = resampled_transactions.std()
std.rename(columns={"price":"std"}, inplace=True)
std.drop(columns="item_id", inplace=True)
std = std.groupby(level=0).tail(1)
std.index = std.index.droplevel("time")
std = std.groupby(level=0).tail(1)

std.head()

,std
item_id,
1,5.762773e+07
2,NaN
3,1.096016e+03
4,6.507919e+02
5,1.622723e+02


In [318]:
median = resampled_transactions.median()
median.rename(columns={"price":"median"}, inplace=True)
median.drop(columns="item_id", inplace=True)
median = median.groupby(level=0).tail(1)
median.index = median.index.droplevel("time")
median = median.groupby(level=0).tail(1)

median.head()

,median
item_id,
1,287000.0
2,439.0
3,1225.0
4,2045.0
5,1500.0


In [319]:
max_price = resampled_transactions.max()
max_price.rename(columns={"price":"max"}, inplace=True)
max_price.drop(columns="item_id", inplace=True)
max_price = max_price.groupby(level=0).tail(1)
max_price.index = max_price.index.droplevel("time")
max_price = max_price.groupby(level=0).tail(1)

max_price.head()

,max
item_id,
1,100000000.0
2,439.0
3,2000.0
4,3480.0
5,1900.0


In [320]:
min_price = resampled_transactions.min()
min_price.rename(columns={"price":"min"}, inplace=True)
min_price.drop(columns="item_id", inplace=True)
min_price = min_price.groupby(level=0).tail(1)
min_price.index = min_price.index.droplevel("time")
min_price = min_price.groupby(level=0).tail(1)

min_price.head()

,min
item_id,
1,85000.0
2,439.0
3,450.0
4,800.0
5,1500.0


In [321]:
quantile_25 = resampled_transactions.agg(lambda x: x.quantile(0.25))
quantile_25.rename(columns={"price":"quantile_25"}, inplace=True)
quantile_25.drop(columns="item_id", inplace=True)
quantile_25 = quantile_25.groupby(level=0).tail(1)
quantile_25.index = quantile_25.index.droplevel("time")
quantile_25 = quantile_25.groupby(level=0).tail(1)

quantile_25.head()

,quantile_25
item_id,
1,186000.0
2,439.0
3,837.5
4,800.0
5,1500.0


In [322]:
quantile_75 = resampled_transactions.agg(lambda x: x.quantile(0.75))
quantile_75.rename(columns={"price":"quantile_75"}, inplace=True)
quantile_75.drop(columns="item_id", inplace=True)
quantile_75 = quantile_75.groupby(level=0).tail(1)
quantile_75.index = quantile_75.index.droplevel("time")
quantile_75 = quantile_75.groupby(level=0).tail(1)

quantile_75.head()

,quantile_75
item_id,
1,50143500.0
2,439.0
3,1612.5
4,2048.0
5,1500.0


In [323]:
items = items.merge(pd.concat([numerosity, mean, std, median, max_price, min_price, quantile_25, quantile_75], axis=1), left_index=True, right_index=True, how="left")
items.head()

,allow_sell,category,cons,craft_pnt,craftable,critical,description,dragon_power,estimate,name,...,datetime,interval,numerosity,mean,std,median,max,min,quantile_25,quantile_75
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,Un foglio di carta senza alcun significato,0,480,Carta,...,2018-02-03 17:17:43,2,3.0,3.345733e+07,5.762773e+07,287000.0,100000000.0,85000.0,186000.0,50143500.0
2,1,0,0,0,0,0,Un bastoncino di legno,0,439,Bastoncino,...,2018-02-03 17:17:43,2,1.0,4.390000e+02,NaN,439.0,439.0,439.0,439.0,439.0
3,1,0,0,0,0,0,Utile per collegare due oggetti,0,423,Colla,...,2018-02-03 17:17:43,2,2.0,1.225000e+03,1.096016e+03,1225.0,2000.0,450.0,837.5,1612.5
5,1,0,0,0,0,0,Cordino poco resistente per collegare due oggetti,0,455,Cordino Fragile,...,2018-02-03 17:17:43,2,88.0,1.581818e+03,1.622723e+02,1500.0,1900.0,1500.0,1500.0,1500.0
7,1,4,1,0,0,0,Una piccola pietra molto leggera,0,404,Pietra Piccola,...,2018-02-03 17:17:43,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Saving market prices to file

In [324]:
items.to_json("data/market_prices.json", orient="records")

## Save items statistics to file

In [327]:
def save_row_to_csv(row):
    if os.path.isfile("items/"+str(row.name)+".csv"):
        data = pd.read_csv("items/"+str(row.name)+".csv")
    else:
        data = pd.DataFrame()
    
    data = data.append(row)
    data.to_csv("items/"+str(row.name)+".csv", index=False)

In [328]:
items.apply(save_row_to_csv, axis=1)

id
1      None
2      None
3      None
5      None
7      None
10     None
12     None
23     None
33     None
112    None
114    None
140    None
228    None
269    None
277    None
281    None
305    None
315    None
362    None
371    None
372    None
385    None
4      None
6      None
8      None
11     None
13     None
14     None
15     None
16     None
       ... 
583    None
584    None
585    None
586    None
587    None
588    None
589    None
590    None
591    None
592    None
593    None
594    None
652    None
676    None
692    None
693    None
694    None
68     None
69     None
70     None
71     None
72     None
73     None
700    None
701    None
702    None
703    None
704    None
705    None
706    None
Length: 751, dtype: object